In [5]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import webbrowser
import tflearn
import tensorflow
import random
import json


In [6]:
with open("intents.json") as file:
    data= json.load(file)
    
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up',
    'how are ya',
    'heyy',
    'whatsup'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'tyyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
    'you are made by',

In [7]:
words=[] #words in dict
labels=[]
docs_x=[] #questions
docs_y=[] #tags for each question

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
    if intent["tag"]  not in labels:
        labels.append(intent["tag"])
words[:3]

['Hi', 'How', 'are']

In [8]:
docs_x[:3]

[['Hi'], ['How', 'are', 'you'], ['Is', 'anyone', 'there', '?']]

In [9]:
labels[:3]

['greeting', 'goodbye', 'creator']

In [10]:
docs_y[:3]

['greeting', 'greeting', 'greeting']

In [11]:
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

In [12]:
words[4:8]

['about', 'act', 'address', 'admid']

In [13]:
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))
labels = sorted(labels)
training = []
output = []
out_empty = [0 for _ in range(len(labels))]


In [14]:
for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    #output of above question
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)
    
training = numpy.array(training)
output = numpy.array(output)

**Output will of same lenght as of total number of tags we have**<br>
For Example :<br>
````
    tags=["hi',"buy","good"] 
    output=[0,0,1]
````
here 1 shows which tags is the output of the following question<br>
***this is achieved by the line***<br> 
```    output_row[labels.index(docs_y[x])] = 1
```

In [15]:
#defining input layer shape
net = tflearn.input_data(shape=[None, len(training[0])])
#2 hidden layer
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
#output layer shape and activation
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)


Training Step: 29709  | total loss: 0.00015 | time: 0.163s
| Adam | epoch: 646 | loss: 0.00015 - acc: 1.0000 -- iter: 312/362
